In [1]:
import os
import pandas as pd
import numpy as np
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import copy
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import joblib
import optuna
from tqdm import tqdm
import time
import codecs
import pickle

# Configuration

In [2]:
rawdata_dir = 'rawdata'
data_dir = 'data'

lags = [1,2,3,6,12,24,36]

# NUM_FOLD_DAYS = 28
NUM_FOLD_DAYS = 56
VALID_FIRST_DAY = 1914
VALID_LAST_DAY = 1941
NUM_USE_DAY = 365
# TRAIN_FIRST_DAY = max(lags)
TRAIN_FIRST_DAY = VALID_FIRST_DAY - NUM_USE_DAY + 1

OPTUNA_TRIAL = 70

# Read data

In [3]:
sales = pd.read_csv(os.path.join(rawdata_dir, 'sales_train_evaluation.csv'))
sales.name = 'sales'
calendar = pd.read_csv(os.path.join(rawdata_dir, 'calendar.csv'))
calendar.name = 'calendar'
prices = pd.read_csv(os.path.join(rawdata_dir, 'sell_prices.csv'))
prices.name = 'prices'

Since, the validation data is now available for the days 1914-1941, Adding zero sales for days: d_1942 - d_1969(Test)

In [4]:
#Add zero sales for the remaining days 1942-1969
for d in range(1942,1970):
    col = 'd_' + str(d)
    sales[col] = 0
    sales[col] = sales[col].astype(np.int16)

# Downcast

In [5]:
#Downcast in order to save memory
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df

sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

# Melt data

In [6]:
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()

## Combine data
Combine price data from prices dataframe and days data from calendar dataset.

In [7]:
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df, prices, on=['store_id','item_id','wm_yr_wk'], how='left')

# Feature Engineering

## Label Encoding

In [8]:
#Store the categories along with their codes
d_id = dict(zip(df.id.cat.codes, df.id))
d_item_id = dict(zip(df.item_id.cat.codes, df.item_id))
d_dept_id = dict(zip(df.dept_id.cat.codes, df.dept_id))
d_cat_id = dict(zip(df.cat_id.cat.codes, df.cat_id))
d_store_id = dict(zip(df.store_id.cat.codes, df.store_id))
d_state_id = dict(zip(df.state_id.cat.codes, df.state_id))

In [9]:
df.d = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
cols = df.dtypes.index.tolist()
types = df.dtypes.values.tolist()
for i,type in enumerate(types):
    if type.name == 'category':
        df[cols[i]] = df[cols[i]].cat.codes
        
df.drop('date',axis=1,inplace=True)

## Introduce lags

In [10]:
#Introduce lags
def make_lags(df, lags):
    for lag in lags:
        df['sold_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)
        
    return df

In [11]:
df = make_lags(df, lags)

## make new features

In [12]:
def make_new_feature(df, d_last = df['d'].max()):
    # copy sold
    sold_tmp = copy.copy(df['sold'])
    
    # embed NaN
    df['sold'][df['d'] > d_last] = np.nan
    
    # make feature using sold
    df['item_sold_avg'] = df.groupby('item_id')['sold'].transform('mean').astype(np.float16)
    df['state_sold_avg'] = df.groupby('state_id')['sold'].transform('mean').astype(np.float16)
    df['store_sold_avg'] = df.groupby('store_id')['sold'].transform('mean').astype(np.float16)
    df['cat_sold_avg'] = df.groupby('cat_id')['sold'].transform('mean').astype(np.float16)
    df['dept_sold_avg'] = df.groupby('dept_id')['sold'].transform('mean').astype(np.float16)
    df['cat_dept_sold_avg'] = df.groupby(['cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
    df['store_item_sold_avg'] = df.groupby(['store_id','item_id'])['sold'].transform('mean').astype(np.float16)
    df['cat_item_sold_avg'] = df.groupby(['cat_id','item_id'])['sold'].transform('mean').astype(np.float16)
    df['dept_item_sold_avg'] = df.groupby(['dept_id','item_id'])['sold'].transform('mean').astype(np.float16)
    df['state_store_sold_avg'] = df.groupby(['state_id','store_id'])['sold'].transform('mean').astype(np.float16)
    df['state_store_cat_sold_avg'] = df.groupby(['state_id','store_id','cat_id'])['sold'].transform('mean').astype(np.float16)
    df['store_cat_dept_sold_avg'] = df.groupby(['store_id','cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
    
    df['rolling_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
    df['expanding_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.expanding(2).mean()).astype(np.float16)
    
    df['daily_avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','d'])['sold'].transform('mean').astype(np.float16)
    df['avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform('mean').astype(np.float16)
    df['selling_trend'] = (df['daily_avg_sold'] - df['avg_sold']).astype(np.float16)
    df.drop(['daily_avg_sold','avg_sold'],axis=1,inplace=True)
    
    # restore sold
    df['sold'] = sold_tmp
    
    return df

In [13]:
df_all = copy.copy(df)
df_all = make_new_feature(df_all)
df_all = df_all[df_all['d'] >= max(lags)]
df_all.info()

df_all.to_pickle(os.path.join(data_dir, 'data_all.pkl'))   # save df_all (use this after CV)
del df_all
gc.collect()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58967660 entries, 1067150 to 60034809
Data columns (total 43 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   id                        int16  
 1   item_id                   int16  
 2   dept_id                   int8   
 3   cat_id                    int8   
 4   store_id                  int8   
 5   state_id                  int8   
 6   d                         int16  
 7   sold                      int16  
 8   wm_yr_wk                  int16  
 9   weekday                   int8   
 10  wday                      int8   
 11  month                     int8   
 12  year                      int16  
 13  event_name_1              int8   
 14  event_type_1              int8   
 15  event_name_2              int8   
 16  event_type_2              int8   
 17  snap_CA                   int8   
 18  snap_TX                   int8   
 19  snap_WI                   int8   
 20  sell_price      

0

# Cross Validation

## Cross Validation for each store (function)

In [14]:
def xgboost_cv_store(df, store):
    def objective(trial):
        # param space
        param = {
            'objective': 'reg:squarederror',
            'n_estimators': 1000,
            'learning_rate': 0.1,
            'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 0.95, 0.05),
            'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.6, 0.95, 0.05),
            'max_depth':  trial.suggest_int('max_depth', 3, 9),
            'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
            'min_child_weight': trial.suggest_loguniform('min_child_weight', 0.1, 10.0),
            'tree_method': 'gpu_hist'
        }
        
        ### START: CV Loop ###
        isBreakCV = False
        fold_iter = 0
        rmse_mean = 0.
        while not(isBreakCV):
            start = time.time()   # start timer
            
            d_valid_last = VALID_LAST_DAY - NUM_FOLD_DAYS * fold_iter
            d_train_last = VALID_LAST_DAY - NUM_FOLD_DAYS * (fold_iter + 1)
            if d_train_last < TRAIN_FIRST_DAY + NUM_FOLD_DAYS * 2:
                isBreakCV = True
            
            # make_new_feature
            df_fold_file = os.path.join(data_dir, 'data_foldday'+str(NUM_FOLD_DAYS)+'_fold'+str(fold_iter + 1)+'.pkl')
            
            if os.path.isfile(df_fold_file):   # make_new_feature 適用後のファイルがあればそれを使う
                df_fold = pd.read_pickle(df_fold_file)
            else:
                df_fold = df[df['d'] <= d_valid_last]
                df_fold = make_new_feature(df_fold, d_train_last)   # train の情報だけを使って特徴量を作成
                df_fold.to_pickle(df_fold_file)   # pickleとして保存しておく

            # Extract store
            df_fold = df_fold[df_fold['store_id'] == store]
                        
            # Split data
            X_train = df_fold[(df_fold['d'] >= TRAIN_FIRST_DAY) & (df_fold['d'] <= d_train_last)].drop('sold', axis=1)
            y_train = df_fold[(df_fold['d'] >= TRAIN_FIRST_DAY) & (df_fold['d'] <= d_train_last)]['sold']
            X_valid = df_fold[(df_fold['d'] > d_train_last) & (df_fold['d'] <= d_valid_last)].drop('sold', axis=1)
            y_valid = df_fold[(df_fold['d'] > d_train_last) & (df_fold['d'] <= d_valid_last)]['sold']
            
            # Train XGBoost model
            model = xgb.XGBRegressor(**param)
#             model.fit(X_train, y_train, eval_set=[(X_train,y_train), (X_valid, y_valid)],
#                  eval_metric='rmse', verbose=10, early_stopping_rounds=100)
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
                  eval_metric='rmse', verbose=10, early_stopping_rounds=50)
            y_pred = model.predict(X_valid)
            rmse_mean += np.sqrt(mean_squared_error(y_valid.values, y_pred))
            
            fold_iter += 1
            elapsed_time = time.time() - start   # stop timer
            print("[{0}] Fold{1}: {2} [sec]".format(d_store_id[store], fold_iter, elapsed_time))
        ### END: CV Loop ###
        
        del df_fold, model, X_train, y_train, X_valid, y_valid, y_pred
        gc.collect()
        
        rmse_mean /= fold_iter
        return rmse_mean
    return objective


## Execute CV

In [16]:
stores = sales.store_id.cat.codes.unique().tolist()

# find best params by doing CV (optuna)
# for store in tqdm(stores[5:]):
for store in tqdm(stores[7:]):
    print('##################### CV START: {0} #####################'.format(d_store_id[store]))
    study = optuna.create_study()
    study.optimize(xgboost_cv_store(df, store), n_trials=OPTUNA_TRIAL)
    
    # save optuna log
    df_trial = study.trials_dataframe()
    df_trial.to_pickle(os.path.join('report', 'OptunaTrials_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'))
    del df_trial
    gc.collect()
    
    # savae best params found by CV
    bestparam_filename = 'BestParamsSelectedByCV_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
    with open(os.path.join('result', 'best_param_cv', bestparam_filename), 'wb') as f:
        pickle.dump(study.best_params, f)


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

##################### CV START: WI_1 #####################
[0]	validation_0-rmse:2.64220
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:2.38514
[20]	validation_0-rmse:2.45707
[30]	validation_0-rmse:4.80370
[40]	validation_0-rmse:4.67954
[50]	validation_0-rmse:4.05469
[60]	validation_0-rmse:3.31458
Stopping. Best iteration:
[13]	validation_0-rmse:2.33653

[WI_1] Fold1: 9.204160213470459 [sec]
[0]	validation_0-rmse:2.67640
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:6.46294
[20]	validation_0-rmse:8.18788
[30]	validation_0-rmse:8.73175
[40]	validation_0-rmse:9.57307
[50]	validation_0-rmse:9.70447
Stopping. Best iteration:
[3]	validation_0-rmse:2.50928

[WI_1] Fold2: 33.47006034851074 [sec]
[0]	validation_0-rmse:2.67790
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:3.49585
[20]	validation_0-rmse:5.85429
[30]	validation_0-rmse:5.75831
[40]	validation_0-rmse:5.47913


[I 2020-06-26 10:23:52,576] Finished trial#0 with value: 2.474556636810303 with parameters: {'subsample': 0.75, 'colsample_bytree': 0.7, 'max_depth': 6, 'gamma': 0.5058658910052087, 'min_child_weight': 6.504364285359612}. Best is trial#0 with value: 2.474556636810303.


[0]	validation_0-rmse:2.64618
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:2.29556
[20]	validation_0-rmse:2.31506
[30]	validation_0-rmse:2.34164
[40]	validation_0-rmse:3.09947
[50]	validation_0-rmse:3.55218
[60]	validation_0-rmse:3.61930
Stopping. Best iteration:
[12]	validation_0-rmse:2.24941

[WI_1] Fold1: 36.01420021057129 [sec]
[0]	validation_0-rmse:2.68349
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:2.45113
[20]	validation_0-rmse:3.54066
[30]	validation_0-rmse:3.43334
[40]	validation_0-rmse:4.02342
[50]	validation_0-rmse:3.31466
[60]	validation_0-rmse:5.07969
Stopping. Best iteration:
[12]	validation_0-rmse:2.36514

[WI_1] Fold2: 34.20914936065674 [sec]
[0]	validation_0-rmse:2.67923
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:3.78723
[20]	validation_0-rmse:4.08348
[30]	validation_0-rmse:4.42102
[40]	validation_0-rmse:4.35710
[50]	validation_0-rmse:6.01

[I 2020-06-26 10:26:36,086] Finished trial#1 with value: 2.417747640609741 with parameters: {'subsample': 0.8, 'colsample_bytree': 0.8, 'max_depth': 7, 'gamma': 1.0467291660029842e-05, 'min_child_weight': 0.318622416502928}. Best is trial#1 with value: 2.417747640609741.


...

[I 2020-06-26 19:42:22,812] Finished trial#67 with value: 3.343220567703247 with parameters: {'subsample': 0.75, 'colsample_bytree': 0.6, 'max_depth': 8, 'gamma': 0.054354353917702415, 'min_child_weight': 8.344229489425164}. Best is trial#56 with value: 3.3210408687591553.


[0]	validation_0-rmse:3.79888
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:10.40373
[20]	validation_0-rmse:20.39078
[30]	validation_0-rmse:20.54750
[40]	validation_0-rmse:22.14497
[50]	validation_0-rmse:25.35716
Stopping. Best iteration:
[3]	validation_0-rmse:3.78620

[WI_3] Fold1: 34.56579613685608 [sec]
[0]	validation_0-rmse:3.94444
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:6.05084
[20]	validation_0-rmse:15.79035
[30]	validation_0-rmse:16.00634
[40]	validation_0-rmse:17.68643
[50]	validation_0-rmse:17.02675
Stopping. Best iteration:
[1]	validation_0-rmse:3.73321

[WI_3] Fold2: 33.11044764518738 [sec]
[0]	validation_0-rmse:3.47230
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:12.85559
[20]	validation_0-rmse:13.77689
[30]	validation_0-rmse:18.58713
[40]	validation_0-rmse:20.06456
[50]	validation_0-rmse:19.26048
Stopping. Best iteration:
[1]	validation_0-rms

[I 2020-06-26 19:45:01,692] Finished trial#68 with value: 3.509158754348755 with parameters: {'subsample': 0.8, 'colsample_bytree': 0.6, 'max_depth': 5, 'gamma': 0.4635168283181346, 'min_child_weight': 1.2400493717593435}. Best is trial#56 with value: 3.3210408687591553.


[0]	validation_0-rmse:3.81158
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:5.09974
[20]	validation_0-rmse:7.74093
[30]	validation_0-rmse:8.69798
[40]	validation_0-rmse:11.04180
[50]	validation_0-rmse:11.51189
Stopping. Best iteration:
[1]	validation_0-rmse:3.66470

[WI_3] Fold1: 34.932260036468506 [sec]
[0]	validation_0-rmse:3.95068
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:6.96971
[20]	validation_0-rmse:13.91893
[30]	validation_0-rmse:16.48680
[40]	validation_0-rmse:19.64792
[50]	validation_0-rmse:19.82228
Stopping. Best iteration:
[1]	validation_0-rmse:3.79629

[WI_3] Fold2: 33.5925829410553 [sec]
[0]	validation_0-rmse:3.50194
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:6.75217
[20]	validation_0-rmse:12.01397
[30]	validation_0-rmse:14.64307
[40]	validation_0-rmse:15.27399
[50]	validation_0-rmse:16.29903
Stopping. Best iteration:
[1]	validation_0-rmse:3.

[I 2020-06-26 19:47:42,684] Finished trial#69 with value: 3.507691526412964 with parameters: {'subsample': 0.75, 'colsample_bytree': 0.65, 'max_depth': 7, 'gamma': 0.04738409623292083, 'min_child_weight': 8.48007341508832}. Best is trial#56 with value: 3.3210408687591553.
100%|███████████████████████████████████████████████████████████████████████████████| 3/3 [9:26:04<00:00, 11321.60s/it]


In [17]:
del df
gc.collect()

22

## Train & Predict by best model (function)

In [19]:
def xgboost_by_bestparam_for_valid(df, store, best_params, valid_preds):
    print('##################### Traing4ValidByBestParam START: {0} #####################'.format(d_store_id[store]))
    print('Best Parameters of store {0} model: {1}'.format(d_store_id[store], best_params))
    
    # prepare data set
    df_store = df[df['store_id'] == store]
    X_train = df_store[df_store['d'] < VALID_FIRST_DAY].drop('sold', axis=1)
    y_train = df_store[df_store['d'] < VALID_FIRST_DAY]['sold']
    X_valid = df_store[(df_store['d'] >= VALID_FIRST_DAY) & (df_store['d'] <= VALID_LAST_DAY)].drop('sold', axis=1)
    
    # train XGBoost by using best_params
    model = xgb.XGBRegressor(**best_params)
    model.fit(X_train, y_train, eval_metric='rmse')
    
    # save model for validation
    modelfile = 'BestModel4valid_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
    with open(os.path.join('result', 'best_model', modelfile), 'wb') as f:
        pickle.dump(model, f)
    
    # predict for valid
    valid_preds[X_valid.index] = model.predict(X_valid)
    del df_store, X_train, y_train, X_valid, model
    gc.collect()
    

In [20]:
def xgboost_by_bestparam_for_test(df, store, best_params, eval_preds):
    print('##################### Traing4TestByBestParam START: {0} #####################'.format(d_store_id[store]))
    
    # prepare data set
    df_store = df[df['store_id'] == store]
    X_train = df_store[df_store['d'] <= VALID_LAST_DAY].drop('sold', axis=1)
    y_train = df_store[df_store['d'] <= VALID_LAST_DAY]['sold']
    X_test = df_store[df_store['d'] > VALID_LAST_DAY].drop('sold', axis=1)
    
    # train XGBoost by using best_params
    model = xgb.XGBRegressor(**best_params)
    model.fit(X_train, y_train, eval_metric='rmse')
        
    # predict for test
    eval_preds[X_test.index] = model.predict(X_test)
    
    # save model for test
    modelfile = 'BestModel4test_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
    with open(os.path.join('result', 'best_model', modelfile), 'wb') as f:
        pickle.dump(model, f)
    
    del df_store, X_train, y_train, X_test, model
    gc.collect()
    

## Train & Predict by best model

In [21]:
# train & predict (best model)
df_all = pd.read_pickle(os.path.join(data_dir, 'data_all.pkl'))

valid = df_all[(df_all['d'] >= VALID_FIRST_DAY) & (df_all['d'] <= VALID_LAST_DAY)][['id','d','sold']]
test = df_all[df_all['d'] > VALID_LAST_DAY][['id','d','sold']]
valid_preds = valid['sold']
eval_preds = test['sold']

for store in tqdm(stores):
    # read best param selected by CV
    bestparam_filename = 'BestParamsSelectedByCV_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
    with open(os.path.join('result', 'best_param_cv', bestparam_filename), 'rb') as f:
        best_params = pickle.load(f)

    # add other params
    best_params['objective'] = 'reg:squarederror'
    best_params['n_estimators'] = 1000
    best_params['learning_rate'] = 0.1
    best_params['tree_method'] = 'gpu_hist'
    
    # best model for valid
    xgboost_by_bestparam_for_valid(df_all, store, best_params, valid_preds)
    
    # best model for test
    xgboost_by_bestparam_for_test(df_all, store, best_params, eval_preds)


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

##################### Traing4ValidByBestParam START: CA_1 #####################
Best Parameters of store CA_1 model: {'subsample': 0.75, 'colsample_bytree': 0.8, 'max_depth': 8, 'gamma': 0.015729391316734863, 'min_child_weight': 0.8876601143413467, 'objective': 'reg:squarederror', 'n_estimators': 1000, 'learning_rate': 0.1, 'tree_method': 'gpu_hist'}
##################### Traing4TestByBestParam START: CA_1 #####################


 10%|████████▏                                                                         | 1/10 [03:20<30:01, 200.13s/it]

##################### Traing4ValidByBestParam START: CA_2 #####################
Best Parameters of store CA_2 model: {'subsample': 0.75, 'colsample_bytree': 0.8, 'max_depth': 6, 'gamma': 0.016578195363804397, 'min_child_weight': 4.896248877904603, 'objective': 'reg:squarederror', 'n_estimators': 1000, 'learning_rate': 0.1, 'tree_method': 'gpu_hist'}
##################### Traing4TestByBestParam START: CA_2 #####################


 20%|████████████████▍                                                                 | 2/10 [05:50<24:41, 185.16s/it]

...

 80%|█████████████████████████████████████████████████████████████████▌                | 8/10 [20:39<05:21, 160.66s/it]

##################### Traing4ValidByBestParam START: WI_2 #####################
Best Parameters of store WI_2 model: {'subsample': 0.6, 'colsample_bytree': 0.8, 'max_depth': 5, 'gamma': 0.003070606752470358, 'min_child_weight': 0.9105773853224478, 'objective': 'reg:squarederror', 'n_estimators': 1000, 'learning_rate': 0.1, 'tree_method': 'gpu_hist'}
##################### Traing4TestByBestParam START: WI_2 #####################


 90%|█████████████████████████████████████████████████████████████████████████▊        | 9/10 [22:47<02:30, 150.94s/it]

##################### Traing4ValidByBestParam START: WI_3 #####################
Best Parameters of store WI_3 model: {'subsample': 0.7, 'colsample_bytree': 0.6, 'max_depth': 8, 'gamma': 0.0035029582625726277, 'min_child_weight': 3.621267424068071, 'objective': 'reg:squarederror', 'n_estimators': 1000, 'learning_rate': 0.1, 'tree_method': 'gpu_hist'}
##################### Traing4TestByBestParam START: WI_3 #####################


100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [26:01<00:00, 156.13s/it]


In [22]:
# save valid_preds
valid_preds_filename = 'ValidPreds_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
with open(os.path.join('result', 'preds', valid_preds_filename), 'wb') as f:
    pickle.dump(valid_preds, f)
    
# save test_preds
eval_preds_filename = 'EvalPreds_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
with open(os.path.join('result', 'preds', eval_preds_filename), 'wb') as f:
    pickle.dump(eval_preds, f)

In [23]:
del df_all
gc.collect()

44

# Make submission

In [26]:
#Set actual equal to false if you want to top in the public leaderboard :P
actual = True
if actual == False:
    #Get the validation results(We already have them as less than one month left for competition to end)
    validation = sales[['id']+['d_' + str(i) for i in range(1914,1942)]]
    validation['id']=pd.read_csv(os.path.join(rawdata_dir, 'sales_train_validation.csv')).id
    validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
else:
    #Get the actual validation results
    valid['sold'] = valid_preds
    validation = valid[['id','d','sold']]
    validation = pd.pivot(validation, index='id', columns='d', values='sold').reset_index()
    validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
    validation.id = validation.id.map(d_id).str.replace('evaluation','validation')

#Get the evaluation results
test['sold'] = eval_preds
evaluation = test[['id','d','sold']]
evaluation = pd.pivot(evaluation, index='id', columns='d', values='sold').reset_index()
evaluation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
#Remap the category id to their respective categories
evaluation.id = evaluation.id.map(d_id)

#Prepare the submission
submit = pd.concat([validation,evaluation]).reset_index(drop=True)

isNegative2Zero = False
if isNegative2Zero:
    submit = submit.set_index("id", drop=True)
    submit[submit < 0] = 0
    submit = submit.reset_index()

submit_file = 'xgboost_cv_UseValidPredsIs'+str(actual)+'_Negative2ZeroIs'+str(isNegative2Zero)+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.csv'
submit.to_csv(os.path.join('submission_point', submit_file), index=False)